In [1]:
from ppgmne_prf.load_data import load_data
from ppgmne_prf.optim import (
    get_abstract_model,
    get_fixed_params,
    get_instance,
    get_solution_data,
    solve_instance,
)
from ppgmne_prf.preprocess import preprocess

In [2]:
df_accidents, dict_stations = load_data()
df_quadrants = preprocess(df_accidents, dict_stations)

2023-09-25 21:42:39.473 | INFO     | ppgmne_prf.load_data:load_data:16 - Load data - Início do carregamento os dados de entrada.
2023-09-25 21:42:39.475 | INFO     | ppgmne_prf.load_data:load_data:18 - Load data (accidents) - Carregando os dados históricos dos acidentes.
2023-09-25 21:42:39.476 | WARNING  | ppgmne_prf.load_data:__load_accidents:44 - Load data (accidents) - Dados carregados da cache.
2023-09-25 21:42:39.614 | INFO     | ppgmne_prf.load_data:load_data:21 - Load data (stations) - Carregando as coordenadas das UOPs e delegacias.
2023-09-25 21:42:39.614 | WARNING  | ppgmne_prf.load_data:__load_stations:132 - Load data (stations) - Dados carregados da cache.
2023-09-25 21:42:39.615 | INFO     | ppgmne_prf.load_data:load_data:26 - Load data - Fim do carregamento os dados de entrada.
2023-09-25 21:42:39.616 | INFO     | ppgmne_prf.preprocess:preprocess:37 - Pre-process - Início do pré-processamento dos dados de entrada.
2023-09-25 21:42:39.616 | INFO     | ppgmne_prf.preproces

In [3]:
df = df_accidents.copy()

In [4]:
df.head()

,id,data_inversa,dia_semana,horario,uf,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,...,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop,ano
0,100027,2018-01-01,segunda-feira,06:00:00,TO,RIO DOS BOIS,Falta de Atenção à Condução,Colisão frontal,Com Vítimas Feridas,Pleno dia,...,1,0,1,2,"-9,35641407","-48,54610294",SPRF-TO,DEL02-TO,UOP02-DEL02-TO,2018
1,100044,2018-01-01,segunda-feira,08:30:00,SE,ITAPORANGA DAJUDA,Não guardar distância de segurança,Colisão traseira,Sem Vítimas,Pleno dia,...,2,0,0,2,"-11,04476007","-37,36498",SPRF-SE,DEL02-SE,UOP01-DEL02-SE,2018
2,100046,2018-01-01,segunda-feira,08:50:00,TO,GUARAI,Falta de Atenção à Condução,Colisão lateral,Sem Vítimas,Pleno dia,...,2,0,0,2,"-8,952295","-48,500107",SPRF-TO,DEL02-TO,UOP02-DEL02-TO,2018
3,100052,2018-01-01,segunda-feira,05:30:00,PA,ELDORADO DO CARAJAS,Ingestão de Álcool,Colisão lateral,Com Vítimas Fatais,Amanhecer,...,1,0,0,2,"-6,345512","-49,389626",SPRF-PA,DEL03-PA,UOP01-DEL03-PA,2018
4,100053,2018-01-01,segunda-feira,10:25:00,BA,TUCANO,Ultrapassagem Indevida,Saída de leito carroçável,Com Vítimas Feridas,Pleno dia,...,0,0,1,1,"-11,08661652","-38,83420181",SPRF-BA,DEL02-BA,DEL10/2-UOP02/BA,2018


In [ ]:
# https://towardsdatascience.com/heres-the-most-efficient-way-to-iterate-through-your-pandas-dataframe-4dad88ac92ee

In [ ]:
df_quadrants.isna().sum()

In [ ]:
# Modelo
dict_params = get_fixed_params(df_quadrants)

In [ ]:
# Cria o modelo abstrato:
model = get_abstract_model(
    a=dict_params["a"],
    u=dict_params["u"],
    s=dict_params["s"],
    dist_max=dict_params["dist_max"],
    dist_matrix=dict_params["dist_matrix"],
    accidents_hist=dict_params["accidents_hist"],
)

In [ ]:
P = [p for p in range(1, 80 + 1)]
Q = [q for q in range(0, 33 + 1)]
R = [(p, q) for p in P for q in Q if q <= p]

for pair in R:
    # Cria a instância:
    instance = get_instance(model, p=pair[0], q=pair[1])

    # Resolve o modelo:
    instance, is_feasible = solve_instance(instance)

    # Extrai os dados da soluçao:
    if is_feasible:
        df_sol = get_solution_data(instance, df_quadrants)